Se cuenta con los siguientes archivos con el registro de la
participación de los distintos jugadores en la liga nacional de fútbol.

Jugador (ID_JUGADOR, Nombre_apellido, fecha_nacimiento,
país)

Estadísticas (ID_PARTIDO, ID_JUGADOR, tiempo_juego,
distancia_recorrida, goles)

Partido (ID_PARTIDO, año, semestre, nro_fecha,
equipo_local, equipo_visitante)

Utilizando el API de RDD de PySpark, se desea obtener un RDD con
el nombre y apellido de los jugadores nacidos en Argentina que
hayan aumentado en más de un 50% el tiempo de juego total de
un semestre al otro. Cada año se juegan dos campeonatos (uno por
semestre) y se deben comparar semestres consecutivos (por ejemplo
2017-2 con 2018-1, o 2018-1 con 2018-2). El tiempo de juego total
para un campeonato es la suma de todo el tiempo jugado en los
partidos de ese campeonato.


In [92]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [93]:
jugador_data = [
    (1,'John Connor',"2000-05-20",'Estados Unidos'),
    (2,'Mike Wazosky',"1980-03-15",'Argentina'),
    (3,'James Sullyvan',"1970-08-30",'Argentina'),
    (4,'Nathan Drake',"1985-04-12", 'Colombia'),
    (5, 'Poco Yo',"1995-11-28",'China')
]

stats_data = [
    (1,1,30,100,1),
    (2,1,60,200,1),
    (3,2,10,20,0),
    (4,2,15,40,2),
    (5,3,20,70,0),
    (6,3,22,79,1),
    (7,4,40,120,1),
    (8,4,80,200,3),
    (9,5,5,6,1),
    (10,5,7,10,0)
]

partido_data = [
    (1,2018,1,"2018-05-20",0,1),
    (2,2018,2,"2018-11-15",1,0),
    (3,2019,1,"2019-04-30",0,1),
    (4,2019,2,"2019-10-12",1,0),
    (5,2016,2,"2016-11-28",0,1),
    (6,2017,1,"2017-05-20",1,0),
    (7,2012,1,"2012-01-15",0,1),
    (8,2015,1,"2015-04-30",1,0),
    (9,2011,2,"2011-10-12",0,1),
    (10,2012,1,"2012-02-28",1,0)
]

jugador = sc.parallelize(jugador_data)
stats = sc.parallelize(stats_data)
partido = sc.parallelize(partido_data)

In [94]:
jugador = jugador.toDF(['ID_JUGADOR','Nombre_apellido','fecha_nacimiento','pais']).withColumn\
('fecha_nacimiento', (col('fecha_nacimiento').cast('date'))).rdd
stats = stats.toDF(['ID_PARTIDO', 'ID_JUGADOR', 'tiempo_juego', 'distancia_recorrida', 'goles']).rdd
partido = partido.toDF(['ID_PARTIDO', 'año', 'semestre', 'nro_fecha', 'equipo_local', 'equipo_visitante']).withColumn\
('nro_fecha', col('nro_fecha').cast('date')).rdd


In [95]:
jugador.take(1)

[Row(ID_JUGADOR=1, Nombre_apellido='John Connor', fecha_nacimiento=datetime.date(2000, 5, 20), pais='Estados Unidos')]

In [96]:
stats.take(1)

[Row(ID_PARTIDO=1, ID_JUGADOR=1, tiempo_juego=30, distancia_recorrida=100, goles=1)]

In [97]:
partido.take(1)

[Row(ID_PARTIDO=1, año=2018, semestre=1, nro_fecha=datetime.date(2018, 5, 20), equipo_local=0, equipo_visitante=1)]

In [98]:
jugadores_argentinos = jugador.filter(lambda x: x.pais == 'Argentina').cache()
stats = stats.map(lambda x: (x[1],(x[0],x[2]))).join(jugadores_argentinos)
stats = stats.map(lambda x: (x[1][0][0],(x[1][0][1],x[1][1])))
stats.collect()

[(3, (10, 'Mike Wazosky')),
 (4, (15, 'Mike Wazosky')),
 (5, (20, 'James Sullyvan')),
 (6, (22, 'James Sullyvan'))]

In [99]:
partidos_mapped = partido.map(lambda x: (x.ID_PARTIDO, (x.año, x.semestre)))
partidos_mapped.collect()

[(1, (2018, 1)),
 (2, (2018, 2)),
 (3, (2019, 1)),
 (4, (2019, 2)),
 (5, (2016, 2)),
 (6, (2017, 1)),
 (7, (2012, 1)),
 (8, (2015, 1)),
 (9, (2011, 2)),
 (10, (2012, 1))]

In [100]:
stats_partidos = partidos_mapped.join(stats).cache()
stats_partidos.collect()

[(3, ((2019, 1), (10, 'Mike Wazosky'))),
 (4, ((2019, 2), (15, 'Mike Wazosky'))),
 (5, ((2016, 2), (20, 'James Sullyvan'))),
 (6, ((2017, 1), (22, 'James Sullyvan')))]

In [104]:
def funcion(x,y):
    if (x[2]==y[2]):
        if (x[1]>y[1]):
            return ((x[0]+y[0])/y[0])>1.5
        elif (x[1]<y[1]):
            return ((y[0]+x[0])/x[0])>1.5
    if (x[2]-1==y[2] & x[1]==1 & y[1]==2):
        return ((x[0]+y[0])/y[0])>1.5
    elif (y[2]-1==x[2] & x[1]==2 & y[1]==1):
        return (y[0]+x[0]/x[0])>1.5
    return False

stats_partidos_mapped = stats_partidos.map(lambda x: (x[1][1][1], (x[1][1][0], x[1][0][1], x[1][0][0])))
mejorados_semestre = stats_partidos_mapped.reduceByKey(lambda x,y: (funcion(x,y))).filter(lambda x: x[1]).map(\
                                             lambda x: (x[0]))
mejorados_semestre.collect()


['Mike Wazosky']